In [1]:
import os
import sys

def add_to_path(new_path: str):
    module_path = os.path.abspath(os.path.join(new_path))
    if module_path not in sys.path:
        sys.path.append(module_path)
    
is_colab = False

if is_colab:
    !git clone https://github.com/pdkary/Karys.git
    !cd Karys && git fetch && git pull
    !cd Karys && pip install -r requirements.txt --quiet
    add_to_path("Karys/")
    from google.colab import drive
    drive.mount("/content/drive")
else:
    !cd ../../ && pip install -r requirements.txt --quiet
    add_to_path("../../")

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\python39\\Scripts\\jupyter.exe' -> 'c:\\python39\\Scripts\\jupyter.exe.deleteme'

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


## Aight we doing HOT DOG / NOT HOT DOG discrimination

In [2]:
from data.configs.ImageDataConfig import ImageDataConfig
from data.configs.RandomDataConfig import RandomDataConfig
from data.wrappers.RandomDataWrapper import RandomDataWrapper
from data.wrappers.ImageDataWrapper import ImageDataWrapper
import numpy as np

if is_colab:
  target_path = "drive/MyDrive/Colab/Seefood/HotDog"
  noise_path = "drive/MyDrive/Colab/Seefood/NotHotDog"
else:
  target_path = "./test_input/target_images"
  noise_path = "./test_input/noise_images"

image_config = ImageDataConfig(image_shape=(64,64,3),image_type=".jpg", features_size=2, preview_rows=2, preview_cols=4)

target_data_wrapper = ImageDataWrapper.load_from_file(target_path, image_config, validation_percentage=0.1)
noise_data_wrapper = ImageDataWrapper.load_from_file(noise_path, image_config, validation_percentage=0.1)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
LOADING FROM ./test_input/target_images/*.jpg
LOADING 249 IMAGES
LOADED 249 IMAGES
LOADING FROM ./test_input/noise_images/*.jpg
LOADING 249 IMAGES
LOADED 249 IMAGES


In [3]:
from models.ModelWrapper import ModelWrapper
from tensorflow.keras.layers import Conv2D, Dense, LeakyReLU, MaxPooling2D, Flatten, Activation, BatchNormalization
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

a = 0.08
disc_layers = [
    Conv2D(64,3, padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(64,3), BatchNormalization(), LeakyReLU(a),
    MaxPooling2D(),
    Conv2D(128,3, padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(128,3), BatchNormalization(), LeakyReLU(a),
    MaxPooling2D(),
    Conv2D(256,3, padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(256,3), BatchNormalization(), LeakyReLU(a),
    MaxPooling2D(),
    Conv2D(512,3, padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(512,3), BatchNormalization(), LeakyReLU(a),
    Conv2D(512,3), BatchNormalization(), LeakyReLU(a),
    MaxPooling2D(),
    Flatten(),
    Dense(512), Activation('relu'),
    Dense(512), Activation('relu'),
    Dense(512), Activation('relu'),
    Dense(image_config.features_size), Activation('softmax'),
]
optimizer = Adam(learning_rate=5e-4)
loss = binary_crossentropy
classifier = ModelWrapper(image_config.image_shape, [image_config.features_size], disc_layers, optimizer, loss, flatten_input=False)
classifier.build()

C:\Python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.0-rc1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 64)        256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 62, 62, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 62, 62, 64)        0     

In [4]:
from plotting.TrainPlotter import TrainPlotter
from trainers.ClassificationTrainer import ClassificationTrainer
import numpy as np

epochs=500
trains_per_test=50
batch_size = 1
batches_per_loop = 1

train_columns = ["Hot Dog Train Loss","Not Hot Dog Train Loss", "Hot Dog Test Loss", "Not Hot Dog Test Loss"]
loss_plot = TrainPlotter(moving_average_size=100,labels=train_columns)
trainer = ClassificationTrainer(classifier, target_data_wrapper, noise_data_wrapper)

# if is_colab:
#   image_output_path = image_path + "/images"
# else:
image_output_path = "./test_output"

t_test_loss = 0
n_test_loss = 0
for i in range(epochs):
  loss_plot.start_epoch()  
  t_loss, n_loss = trainer.train(batch_size, batches_per_loop)

  if i % trains_per_test == 0 and i != 0:
    t_test_loss, n_test_loss = trainer.test(6,1)
    test_output_filename = image_output_path + "/train-" + str(i) + ".jpg"
    target_data_wrapper.save_classified_images(test_output_filename, trainer.most_recent_target_output, trainer.most_recent_noise_output)
      
  loss_plot.batch_update([t_loss,n_loss,t_test_loss,n_test_loss])
  loss_plot.log_epoch()

<IPython.core.display.Javascript object>

KeyboardInterrupt: 